In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# --- IMPORTS ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.metrics import r2_score, mean_squared_error
# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, callbacks, backend as K
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import ParameterGrid
# Standard Libraries
import datetime
import warnings
import traceback
from collections import defaultdict
import os
import random # For setting seeds
import time # For timing VI

# --- WARNINGS CONFIGURATION ---
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="Mean of empty slice")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in log")
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*Converge.*") # Ignore convergence warnings generally
pd.options.mode.chained_assignment = None

# --- TENSORFLOW/KERAS CONFIGURATION ---
# Set random seeds for reproducibility (optional but recommended)
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' # TF 2.x
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# Configure GPU memory growth if available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e) # Memory growth must be set before GPUs have been initialized

# --------------------------------------------------------------------------
# FUNCTION DEFINITIONS (Steps 1 - 3 are identical to GLM+H)
# --------------------------------------------------------------------------

# Step 1: Load and Prepare Dataset (Identical to GLM+H version)
def load_prepare_data(file_path):
    """ Loads, cleans, calculates returns/market cap, handles dates/IDs. """
    print(f"Laster data fra: {file_path}")
    try:
        df = pd.read_csv(file_path, low_memory=False)
    except FileNotFoundError: print(f"FEIL: Fil '{file_path}' ikke funnet."); return None
    print(f"Data lastet inn. Form: {df.shape}")
    date_col = 'Date' if 'Date' in df.columns else 'eom'
    id_col = 'Instrument' if 'Instrument' in df.columns else 'id'
    price_col = 'ClosePrice' if 'ClosePrice' in df.columns else 'prc'
    shares_col = 'CommonSharesOutstanding'
    rf_col = 'NorgesBank10Y'
    sector_col = 'EconomicSector'
    if date_col not in df.columns: print("FEIL: Dato-kolonne mangler."); return None
    if id_col not in df.columns: print("FEIL: Instrument-ID mangler."); return None
    if price_col not in df.columns: print(f"FEIL: Pris-kolonne ('{price_col}') mangler."); return None
    df = df.rename(columns={date_col: 'Date', id_col: 'Instrument'})
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=["Instrument", "Date"]).reset_index(drop=True)
    print("Dato konvertert og data sortert.")
    df["MonthlyReturn"] = df.groupby("Instrument")[price_col].pct_change()
    df["MonthlyReturn"].fillna(0, inplace=True)
    df["MonthlyReturn"] = winsorize(df["MonthlyReturn"].values, limits=[0.01, 0.01])
    print("Månedlig avkastning ('MonthlyReturn') beregnet/winsorisert.")
    if rf_col not in df.columns: df[rf_col] = 0; print(f"ADVARSEL: '{rf_col}' mangler, bruker 0.")
    df["MonthlyRiskFreeRate_t"] = df[rf_col] / 12 / 100 if df[rf_col].abs().max() > 1 else df[rf_col] / 12
    df["TargetReturn_t"] = df["MonthlyReturn"] - df["MonthlyRiskFreeRate_t"] # Model target y
    print("Risikojustert avkastning ('TargetReturn_t') beregnet (modellens y).")
    df['NextMonthlyReturn_t+1'] = df.groupby('Instrument')['MonthlyReturn'].shift(-1)
    print("Neste måneds rå avkastning ('NextMonthlyReturn_t+1') beregnet.")
    if shares_col not in df.columns: print(f"FEIL: '{shares_col}' mangler for MarketCap."); return None
    df["MarketCap"] = df[price_col] * df[shares_col]
    df['MarketCap_orig'] = df['MarketCap'].copy() # Save before filling NA or logging
    df['MarketCap'] = df['MarketCap'].fillna(0)
    print("Markedsverdi ('MarketCap') beregnet.")
    if sector_col in df.columns:
        df = pd.get_dummies(df, columns=[sector_col], prefix="Sector", dtype=int)
        print("Sektor dummy-variabler opprettet.")
    df.columns = df.columns.str.replace(" ", "_").str.replace("-", "_")
    print("Kolonnenavn renset.")
    print("Log-transformerer spesifikke variabler...")
    vars_to_log = ["MarketCap", "BM", "ClosePrice", "Volume", "CommonSharesOutstanding"]
    vars_to_log = [v if v in df.columns else v.lower() for v in vars_to_log]
    vars_to_log = [v if v in df.columns else 'prc' if v == 'closeprice' else v for v in vars_to_log]
    vars_to_log = [col for col in vars_to_log if col in df.columns]
    for var in vars_to_log:
        if var in df.columns:
             if pd.api.types.is_numeric_dtype(df[var]):
                 df[f"{var}_positive"] = df[var].where(df[var] > 1e-9, np.nan)
                 df[f"log_{var}"] = np.log(df[f"{var}_positive"])
                 log_median = df[f"log_{var}"].median()
                 df[f"log_{var}"] = df[f"log_{var}"].fillna(log_median)
                 if pd.isna(log_median): df[f"log_{var}"] = df[f"log_{var}"].fillna(0)
                 df.drop(columns=[f"{var}_positive"], inplace=True)
             else: print(f"  ADVARSEL: Kolonne '{var}' er ikke numerisk, hopper over log-transformasjon.")
    print("Log-transformasjon fullført.")
    if 'MarketCap_orig' not in df.columns and 'MarketCap' in df.columns: df['MarketCap_orig'] = df['MarketCap'].copy()
    return df

# Step 1.5: Rank Standardization (Identical to GLM+H version)
def rank_standardize_features(df, features_to_standardize):
    """ Performs rank standardization cross-sectionally to [-1, 1]. """
    print(f"Rank standardiserer {len(features_to_standardize)} features...")
    if 'Date' not in df.columns: print("FEIL: 'Date' mangler."); return df
    features_present = [f for f in features_to_standardize if f in df.columns]
    if len(features_present) < len(features_to_standardize):
        missing = [f for f in features_to_standardize if f not in features_present]
        print(f"  ADVARSEL: Følgende features manglet for standardisering: {missing}")
    if not features_present: print("  Ingen features å standardisere."); return df
    def rank_transform(x): x_numeric = pd.to_numeric(x, errors='coerce'); ranks = x_numeric.rank(pct=True); return ranks * 2 - 1
    try:
        ranked_cols = df.groupby('Date')[features_present].transform(rank_transform)
        df[features_present] = ranked_cols
    except Exception as e:
        print(f"  ADVARSEL under rank standardisering (transform): {e}. Prøver alternativ apply (saktere).")
        try:
            df_std = df.set_index('Date')
            for col in features_present: df_std[col] = df_std.groupby(level=0)[col].apply(rank_transform)
            df = df_std.reset_index()
        except Exception as e2: print(f"  FEIL: Også alternativ standardisering feilet: {e2}"); return df
    print("Rank standardisering fullført.")
    return df

# Step 2: Define Feature Sets (Identical to GLM+H version)
def define_features(df):
    """ Identifies numeric predictor features from DataFrame 'df'. """
    print("Identifiserer numeriske features...")
    if df is None or df.empty: print("  FEIL: DataFrame er tom."); return []
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    print(f"  Funnet {len(numeric_cols)} numeriske kolonner totalt.")
    cols_to_exclude = ['Instrument', 'Date', 'level_0', 'index', 'Year', 'MonthYear', 'TargetReturn_t', 'NextMonthlyReturn_t+1', 'MonthlyReturn', 'MonthlyRiskFreeRate_t', 'MarketCap_orig', 'rank', 'DecileRank', 'eq_weights', 'me_weights']
    cols_to_exclude.extend([col for col in df.columns if 'return_stock' in col or 'return_portfolio' in col])
    log_cols = [col for col in numeric_cols if col.startswith('log_')]
    originals_of_log = [col.replace('log_','') for col in log_cols]
    common_originals = ['MarketCap', 'ClosePrice', 'prc', 'Volume', 'CommonSharesOutstanding', 'BM']
    originals_to_exclude = [orig for orig in originals_of_log if orig in df.columns and orig in common_originals]
    cols_to_exclude.extend(originals_to_exclude)
    cols_to_exclude = list(set(cols_to_exclude))
    potential_features = [col for col in numeric_cols if col not in cols_to_exclude]
    final_features = []
    for col in potential_features:
        if col in df.columns:
            if df[col].nunique(dropna=True) > 1 and df[col].std(ddof=0, skipna=True) > 1e-9: final_features.append(col)
    final_features = sorted(list(set(final_features)))
    print(f"  Identifisert {len(final_features)} features for bruk i modellen.")
    return final_features

# Step 3: Handle Missing / Infinite Values (Identical to GLM+H version)
def clean_data(df, numeric_features_to_impute, essential_cols_for_dropna, target="TargetReturn_t"):
    """ Handles inf/NaN after standardization. Filters essential cols and MarketCap_orig <= 0. """
    print("Starter datarensing (missing/inf)...")
    initial_rows = len(df)
    features_present = [f for f in numeric_features_to_impute if f in df.columns]
    if not features_present: print("  Ingen features for imputering.");
    else:
        inf_mask = df[features_present].isin([np.inf, -np.inf])
        if inf_mask.any().any(): print(f"  Erstatter inf med NaN i {inf_mask.any(axis=0).sum()} feature kolonner..."); df[features_present] = df[features_present].replace([np.inf, -np.inf], np.nan)
        nan_counts_before = df[features_present].isnull().sum(); medians = df[features_present].median(skipna=True); df[features_present] = df[features_present].fillna(medians); nan_counts_after = df[features_present].isnull().sum()
        imputed_cols = (nan_counts_before - nan_counts_after); print(f"  NaNs imputert med median i {imputed_cols[imputed_cols > 0].count()} feature kolonner.")
        if medians.isnull().any(): cols_nan_median = medians[medians.isnull()].index.tolist(); print(f"  ADVARSEL: Median var NaN for: {cols_nan_median}. Fyller resterende NaNs med 0."); df[cols_nan_median] = df[cols_nan_median].fillna(0)
    essential_cols_present = [col for col in essential_cols_for_dropna if col in df.columns]
    for col in [target,'NextMonthlyReturn_t+1','MarketCap_orig']:
         if col in df.columns and col not in essential_cols_present: essential_cols_present.append(col)
    unique_essential_cols = sorted(list(set(essential_cols_present)))
    if unique_essential_cols:
        rows_before_dropna = len(df); df = df.dropna(subset=unique_essential_cols); rows_dropped = rows_before_dropna - len(df)
        if rows_dropped > 0: print(f"  Fjernet {rows_dropped} rader pga. NaN i essensielle kolonner: {unique_essential_cols}")
        else: print(f"  Ingen rader fjernet pga NaN i essensielle kolonner.")
    mc_orig_col = 'MarketCap_orig'
    if mc_orig_col in df.columns:
        rows_before_mc_filter = len(df); df = df[df[mc_orig_col] > 0]; rows_dropped_mc = rows_before_mc_filter - len(df)
        if rows_dropped_mc > 0: print(f"  Fjernet {rows_dropped_mc} rader der {mc_orig_col} <= 0.")
        else: print(f"  Ingen rader fjernet pga {mc_orig_col} <=0.")
    else: print(f"  Advarsel: Kolonnen '{mc_orig_col}' ikke funnet for filtrering.")
    final_rows = len(df); print(f"Datarensing fullført. Form: {df.shape}. Fjernet totalt {initial_rows - final_rows} rader.")
    if df.empty: print("FEIL: Ingen data igjen etter rensing.")
    return df

# --------------------------------------------------------------------------
# Step 4: Yearly Rolling Window Splits (Copied from GLM+H)
# --------------------------------------------------------------------------
def get_yearly_rolling_splits(df, initial_train_years, val_years, test_years=1):
    """ Creates yearly rolling window splits with expanding training and rolling validation. """
    if "Date" not in df.columns: raise ValueError("'Date'-kolonnen mangler i DataFrame.")
    df['Year'] = df["Date"].dt.year; unique_years = sorted(df["Year"].unique()); n_unique_years = len(unique_years)
    print(f"Unike år i data: {n_unique_years} ({unique_years[0]} - {unique_years[-1]})")
    if n_unique_years < initial_train_years + val_years + test_years:
        df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError(f"Ikke nok unike år ({n_unique_years}) for vindu InitTrain={initial_train_years}, Val={val_years}, Test={test_years}.")
    first_test_year_index = initial_train_years + val_years
    if first_test_year_index >= n_unique_years: df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError("InitTrain+Val er for lang for datasettet.")
    first_test_year = unique_years[first_test_year_index]; last_test_year = unique_years[-test_years]
    num_windows = last_test_year - first_test_year + 1
    if num_windows <= 0: df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError("Beregnet antall vinduer er null/negativt.")
    print(f"Genererer {num_windows} årlige rullerende vinduer..."); print(f"(Første testår: {first_test_year}, Siste testår: {last_test_year})")
    for i in range(num_windows):
        current_test_start_year = first_test_year + i; current_test_end_year = current_test_start_year + test_years - 1
        current_val_end_year = current_test_start_year - 1; current_val_start_year = current_val_end_year - val_years + 1
        current_train_end_year = current_val_start_year - 1; current_train_start_year = unique_years[0]
        train_idx = df[(df['Year'] >= current_train_start_year) & (df['Year'] <= current_train_end_year)].index
        val_idx = df[(df['Year'] >= current_val_start_year) & (df['Year'] <= current_val_end_year)].index
        test_idx = df[(df['Year'] >= current_test_start_year) & (df['Year'] <= current_test_end_year)].index
        train_dates = df.loc[train_idx, "Date"].agg(['min', 'max']) if not train_idx.empty else None
        val_dates = df.loc[val_idx, "Date"].agg(['min', 'max']) if not val_idx.empty else None
        test_dates = df.loc[test_idx, "Date"].agg(['min', 'max']) if not test_idx.empty else None
        print(f"\n  Vindu {i+1}/{num_windows}:"); print(f"    Train: {current_train_start_year}-{current_train_end_year} ({len(train_idx)} obs)"); print(f"    Val  : {current_val_start_year}-{current_val_end_year} ({len(val_idx)} obs)"); print(f"    Test : {current_test_start_year}-{current_test_end_year} ({len(test_idx)} obs)")
        yield train_idx, val_idx, test_idx, train_dates, val_dates, test_dates
    df.drop(columns=['Year'], inplace=True, errors='ignore') # Clean up temp col

# --------------------------------------------------------------------------
# Step 5: Neural Network Specific Functions (Adapted for Yearly Flow)
# --------------------------------------------------------------------------

def build_nn_model(input_shape, nn_config, lambda1):
    """Builds a Keras Sequential NN model based on the config."""
    model = keras.Sequential(name=nn_config['name'])
    model.add(layers.Input(shape=(input_shape,)))
    for units in nn_config['hidden_units']:
        model.add(layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l1(lambda1)))
        # Optional: Add BatchNormalization: model.add(layers.BatchNormalization())
    model.add(layers.Dense(1, activation='linear'))
    return model

def run_nn_on_window(X_train, y_train, X_val, y_val, X_test, y_test,
                     nn_config, param_grid, epochs=100, batch_size=10000,
                     patience=5, ensemble_size=10):
    """
    Trains an ensembled NN with tuning for a SINGLE YEARLY WINDOW.
    Tunes lambda1 and learning_rate based on validation set performance.
    Returns OOS predictions, metrics, and optimal hyperparameters found for the window.
    """
    model_name = nn_config['name']
    optim_param_found = None; optimal_lambda1 = np.nan; optimal_lr = np.nan
    best_val_mse = np.inf
    input_shape = X_train.shape[1]
    preds_oos = np.full(y_test.shape[0], np.nan) # Initialize OOS predictions
    r2_oos, mse_oos, sharpe_oos, r2_is_train_val = (np.nan,) * 4 # Initialize metrics

    # Define Keras Callbacks for tuning phase
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=0)
    terminate_nan = callbacks.TerminateOnNaN()

    # print(f"    Tuning {model_name} (Ensemble={ensemble_size}, {len(param_grid)} combos)...") # Verbose
    if X_val.shape[0] < 2:
        print(f"    ADVARSEL ({model_name}): Val set too small (<2 obs). Skipping tuning/training for this window.");
        return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr

    # --- Hyperparameter Tuning Loop (on Train/Val) ---
    for params in param_grid:
        lambda1 = params['lambda1']; learning_rate = params['learning_rate']
        val_preds_ensemble = []
        # print(f"      Testing LR={learning_rate}, L1={lambda1}") # More verbose
        try:
            # --- Ensemble Loop for Validation ---
            for ens_idx in range(ensemble_size):
                K.clear_session(); tf.random.set_seed(SEED + ens_idx) # Consistent seed for tuning ensemble
                model_val = build_nn_model(input_shape, nn_config, lambda1)
                optimizer = Adam(learning_rate=learning_rate) # Use Adam defaults
                model_val.compile(optimizer=optimizer, loss='mean_squared_error')

                # Train on Training Data, Validate on Validation Data
                history = model_val.fit(X_train, y_train, validation_data=(X_val, y_val),
                                        epochs=epochs, batch_size=batch_size,
                                        callbacks=[early_stopping, terminate_nan], verbose=0) # Suppress epoch logs

                # Store validation predictions only if training didn't fail (NaN loss)
                if not np.isnan(history.history['val_loss']).any():
                     val_preds_ensemble.append(model_val.predict(X_val, batch_size=batch_size).flatten())
                else:
                     # print(f"      WARN ({model_name}): NaN loss detected during validation training for LR={learning_rate}, L1={lambda1}") # Verbose
                     val_preds_ensemble = []; break # Discard ensemble if any member fails

            if not val_preds_ensemble: continue # Skip if ensemble failed for these params

            # Average predictions across the ensemble for validation
            avg_val_preds = np.mean(np.array(val_preds_ensemble), axis=0)
            current_val_mse = mean_squared_error(y_val[np.isfinite(avg_val_preds)], avg_val_preds[np.isfinite(avg_val_preds)]) # Handle potential NaNs in preds

            # Update best parameters if current MSE is better
            if not np.isnan(current_val_mse) and current_val_mse < best_val_mse:
                best_val_mse = current_val_mse
                optim_param_found = params
                # print(f"      New best val_mse: {best_val_mse:.6f} with LR={learning_rate}, L1={lambda1}") # Verbose

        except Exception as e:
            # print(f"      ERROR during tuning ensemble for LR={learning_rate}, L1={lambda1}: {e}") # Verbose
            continue # Ignore errors during tuning for specific param set

    if optim_param_found is None:
        print(f"    FEIL ({model_name}): Tuning feilet (ingen gyldig parameter funnet). Skipping final training.")
        return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr

    optimal_lambda1 = optim_param_found['lambda1']; optimal_lr = optim_param_found['learning_rate']
    # print(f"    Optimal params funnet ({model_name}): LR={optimal_lr}, L1={optimal_lambda1} (Val MSE: {best_val_mse:.6f})") # Verbose

    # --- Final Training (on Train+Val) & Prediction (on Test) ---
    # print(f"    Training final {model_name} ensemble with optimal params...") # Verbose
    X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
    test_preds_ensemble = [] # Store OOS predictions from each ensemble member
    is_preds_ensemble = []   # Store IS predictions from each ensemble member (for IS R2)

    try:
        # --- Ensemble Loop for Final Training and Prediction ---
        for i in range(ensemble_size):
            K.clear_session(); tf.random.set_seed(SEED + i + ensemble_size) # Use different seeds from tuning
            model_final = build_nn_model(input_shape, nn_config, optimal_lambda1)
            optimizer = Adam(learning_rate=optimal_lr)
            model_final.compile(optimizer=optimizer, loss='mean_squared_error')

            # Train on combined set. Stop based on epochs. Use TerminateOnNaN callback.
            history_final = model_final.fit(X_train_val, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan], verbose=0)

            if not np.isnan(history_final.history['loss']).any():
                 # Predict OOS (if test set exists)
                 if X_test.shape[0] > 0:
                     test_preds_ensemble.append(model_final.predict(X_test, batch_size=batch_size).flatten())
                 # Predict IS (Train+Val)
                 is_preds_ensemble.append(model_final.predict(X_train_val, batch_size=batch_size).flatten())
            else:
                 # print(f"      WARN ({model_name}): NaN loss detected during FINAL training for ensemble member {i}") # Verbose
                 test_preds_ensemble = []; is_preds_ensemble = []; break # Discard entire final ensemble if one member fails

        # Check if final ensemble training succeeded
        if X_test.shape[0] > 0 and not test_preds_ensemble: raise ValueError(f"Final ensemble training failed ({model_name}) or test set was empty.")
        if not is_preds_ensemble: raise ValueError(f"Final IS prediction failed ({model_name}).")

        # Average predictions across the final ensemble
        if X_test.shape[0] > 0:
            preds_oos = np.mean(np.array(test_preds_ensemble), axis=0)
        preds_is = np.mean(np.array(is_preds_ensemble), axis=0)

    except Exception as e:
        print(f"  FEIL final {model_name} fit/predict: {e}")
        preds_oos.fill(np.nan) # Ensure failure results in NaNs
        r2_oos, mse_oos, sharpe_oos, r2_is_train_val = (np.nan,) * 4
        # Return NaNs for metrics and optimal params on failure
        return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr

    # --- Calculate OOS Metrics ---
    if X_test.shape[0] > 0:
        nan_preds_oos_mask = ~np.isfinite(preds_oos); preds_oos[nan_preds_oos_mask] = 0 # Replace non-finite OOS preds with 0
        valid_oos_mask = np.isfinite(y_test) & np.isfinite(preds_oos) # Use original y_test
        y_test_valid = y_test[valid_oos_mask]; preds_oos_valid = preds_oos[valid_oos_mask]
        if len(preds_oos_valid) > 1:
            # OOS R2 (Gu et al. definition: 1 - SSE/SST0)
            ss_res_oos = np.sum((y_test_valid - preds_oos_valid)**2)
            ss_tot_oos = np.sum(y_test_valid**2) # Use y_test_valid for SST0
            r2_oos = 1 - (ss_res_oos / ss_tot_oos) if ss_tot_oos > 1e-9 else np.nan
            mse_oos = mean_squared_error(y_test_valid, preds_oos_valid)
            # OOS pseudo-Sharpe (simple mean/std of predictions, annualized)
            pred_std_oos = np.std(preds_oos_valid)
            sharpe_oos = (np.mean(preds_oos_valid)/pred_std_oos)*np.sqrt(12) if pred_std_oos > 1e-9 else np.nan

    # --- Calculate IS Metrics (On Train+Val) ---
    nan_preds_is_mask = ~np.isfinite(preds_is); preds_is[nan_preds_is_mask] = 0 # Replace non-finite IS preds with 0
    valid_is_mask = np.isfinite(y_train_val) & np.isfinite(preds_is) # Use original y_train_val
    y_train_val_valid = y_train_val[valid_is_mask]; preds_is_valid = preds_is[valid_is_mask]
    if len(preds_is_valid) > 1:
        # IS R2 (Gu et al. definition: 1 - SSE/SST0)
        ss_res_is = np.sum((y_train_val_valid - preds_is_valid)**2)
        ss_tot_is = np.sum(y_train_val_valid**2) # Use y_train_val_valid for SST0
        r2_is_train_val = 1 - (ss_res_is / ss_tot_is) if ss_tot_is > 1e-9 else np.nan

    # Return the ensembled OOS predictions, metrics, and optimal params for this window
    return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr


# -----------------------------------------------------------------------------
# Step 6.5: Detailed Portfolio Analysis Function (Copied from GLM+H)
# -----------------------------------------------------------------------------
def MDD(returns):
    """ Calculates Maximum Drawdown using arithmetic returns for NAV. """
    returns = pd.Series(returns).fillna(0)
    if returns.empty: return np.nan
    nav = (1 + returns).cumprod(); hwm = nav.cummax(); dd = nav / hwm - 1
    return dd.min() if not dd.empty else np.nan

def perform_detailed_portfolio_analysis(results_df, original_df_subset, benchmark_file=None, ff_factor_file=None, filter_small_caps=False, model_name_label="NN"): # <-- Generic Default Label
    """ Performs detailed portfolio analysis, generates tables and plots for the given model label. (GLM+H Version)"""
    print(f"\n--- Starter Detaljert Porteføljeanalyse ({model_name_label}, Prediction-Sorted Deciles) ---")
    # Correctly format prediction column name based on model label (e.g., yhat_nn1, yhat_nn5)
    pred_col = f'yhat_{model_name_label.lower().replace("-","_").replace("+","h")}'
    ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long = (pd.DataFrame(),)*6 # Init empty dfs

    if pred_col not in results_df.columns: print(f"FEIL: Prediksjonskolonne '{pred_col}' mangler i results_df for {model_name_label}."); return ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long
    required_cols = ['Date', 'Instrument', 'MarketCap_orig', 'NextMonthlyReturn_t+1', 'MonthlyReturn', 'MonthlyRiskFreeRate_t']
    if not all(c in original_df_subset.columns for c in required_cols): print(f"FEIL: Mangler påkrevde kolonner i original_df_subset: { [c for c in required_cols if c not in original_df_subset.columns] }"); return ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long
    portfolio_data = pd.merge(results_df[['Date', 'Instrument', 'TargetReturn_t', pred_col]], original_df_subset[required_cols], on=['Date', 'Instrument'], how='inner')
    portfolio_data = portfolio_data.rename(columns={'TargetReturn_t': 'y_true_t', pred_col: 'yhat_t+1', 'MarketCap_orig': 'me', 'NextMonthlyReturn_t+1': 'ret_t+1'})
    portfolio_data['MonthYear'] = portfolio_data['Date'].dt.to_period('M')
    # Calculate Next Month's Risk-Free Rate using shift(-1) on the mean RF rate for the current month t
    monthly_rf_map = portfolio_data.groupby('MonthYear')['MonthlyRiskFreeRate_t'].mean().shift(-1)
    portfolio_data['NextMonthRiskFreeRate_t+1'] = portfolio_data['MonthYear'].map(monthly_rf_map)

    # Drop rows where essential data for portfolio return calculation is missing
    cols_for_eval_dropna = ['yhat_t+1', 'ret_t+1', 'me', 'NextMonthRiskFreeRate_t+1']
    initial_rows_port = len(portfolio_data); portfolio_data = portfolio_data.dropna(subset=cols_for_eval_dropna); rows_dropped_port = initial_rows_port - len(portfolio_data)
    if rows_dropped_port > 0: print(f"  Fjernet {rows_dropped_port} rader med manglende verdier for porteføljeevaluering.")
    if portfolio_data.empty: print(f"  FEIL: Ingen data igjen for porteføljeanalyse ({model_name_label})."); return ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long
    # Calculate excess return using the next month's risk-free rate
    portfolio_data['excess_ret_t+1'] = portfolio_data['ret_t+1'] - portfolio_data['NextMonthRiskFreeRate_t+1']

    print("  Sorterer aksjer i desiler basert på prediksjon og beregner månedlige porteføljevekter...")
    monthly_data_dict = {}; all_decile_dfs = {i: [] for i in range(10)}; unique_months = sorted(portfolio_data['MonthYear'].unique())
    for month in unique_months:
        monthly_df = portfolio_data[portfolio_data['MonthYear'] == month].copy()
        if filter_small_caps:
            if 'me' in monthly_df.columns and len(monthly_df) > 10: mc_cutoff = monthly_df['me'].quantile(0.10); monthly_df = monthly_df[monthly_df['me'] >= mc_cutoff].copy()
        if len(monthly_df) < 10: continue # Need enough stocks for deciles
        monthly_df = monthly_df.sort_values('yhat_t+1')
        try:
            # Use rank(method='first') to handle ties consistently before qcut
            monthly_df['rank'] = monthly_df['yhat_t+1'].rank(method='first')
            monthly_df['DecileRank'] = pd.qcut(monthly_df['rank'], 10, labels=False, duplicates='drop')
            # Ensure we actually got 10 distinct deciles after dropping duplicates
            if monthly_df['DecileRank'].nunique() < 10: continue
        except ValueError: continue # Skip month if qcut fails (e.g., not enough unique prediction values)
        monthly_df = monthly_df.drop(columns=['rank']); monthly_df["eq_weights"] = 1 / monthly_df.groupby('DecileRank')["Instrument"].transform('size'); monthly_df["me_weights"] = monthly_df["me"] / monthly_df.groupby('DecileRank')["me"].transform('sum'); monthly_df["me_weights"] = monthly_df["me_weights"].fillna(0); monthly_data_dict[month] = monthly_df
        for decile_rank, group_df in monthly_df.groupby('DecileRank'):
            if decile_rank in all_decile_dfs: all_decile_dfs[decile_rank].append(group_df)
    decile_portfolios = {j: pd.concat(all_decile_dfs[j], ignore_index=True) if all_decile_dfs[j] else pd.DataFrame() for j in range(10)}
    if not any(not df.empty for df in decile_portfolios.values()): print(f"  FEIL: Ingen desilporteføljer konstruert ({model_name_label})."); return ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long

    print("  Beregner månedlige og aggregerte desil-metrikker...")
    decile_results = []; monthly_agg_data = {}
    for j in range(10): # Iterate through decile ranks 0 to 9
        rank_df = decile_portfolios.get(j, pd.DataFrame());
        if rank_df.empty: continue
        # Calculate contribution of each stock to the portfolio return for this decile
        rank_df['excess_return_stock_ew']=rank_df["excess_ret_t+1"]*rank_df["eq_weights"]; rank_df['excess_return_stock_vw']=rank_df["excess_ret_t+1"]*rank_df["me_weights"]
        rank_df['pred_excess_return_stock_ew']=rank_df["yhat_t+1"]*rank_df["eq_weights"]; rank_df['pred_excess_return_stock_vw']=rank_df["yhat_t+1"]*rank_df["me_weights"]
        rank_df['return_stock_ew']=rank_df["ret_t+1"]*rank_df["eq_weights"]; rank_df['return_stock_vw']=rank_df["ret_t+1"]*rank_df["me_weights"]
        # Aggregate monthly returns for this decile portfolio
        monthly_rank_j = rank_df.groupby('MonthYear').agg(
            excess_return_portfolio_ew=('excess_return_stock_ew','sum'), excess_return_portfolio_vw=('excess_return_stock_vw','sum'),
            pred_excess_return_portfolio_ew=('pred_excess_return_stock_ew','sum'), pred_excess_return_portfolio_vw=('pred_excess_return_stock_vw','sum'),
            return_portfolio_ew=('return_stock_ew','sum'), return_portfolio_vw=('return_stock_vw','sum') # Sum of raw weighted returns
        ).reset_index()
        monthly_rank_j['DecileRank'] = j; monthly_agg_data[j] = monthly_rank_j # Store monthly results

        # Calculate overall metrics for this decile
        ew_mean_ret=monthly_rank_j["excess_return_portfolio_ew"].mean(); vw_mean_ret=monthly_rank_j["excess_return_portfolio_vw"].mean()
        ew_mean_pred=monthly_rank_j["pred_excess_return_portfolio_ew"].mean(); vw_mean_pred=monthly_rank_j["pred_excess_return_portfolio_vw"].mean()
        # Sharpe Ratio uses Std Dev of RAW returns in denominator
        std_ew_raw=monthly_rank_j["return_portfolio_ew"].std(); std_vw_raw=monthly_rank_j["return_portfolio_vw"].std()
        sharpe_ew=(ew_mean_ret/std_ew_raw)*np.sqrt(12) if std_ew_raw>1e-9 else np.nan; sharpe_vw=(vw_mean_ret/std_vw_raw)*np.sqrt(12) if std_vw_raw>1e-9 else np.nan
        decile_results.append({'DecileRank':j,'ew_mean_pred':ew_mean_pred,'ew_mean_ret':ew_mean_ret,'std_ew_ret':std_ew_raw,'sharpe_ew':sharpe_ew,'vw_mean_pred':vw_mean_pred,'vw_mean_ret':vw_mean_ret,'std_vw_ret':std_vw_raw,'sharpe_vw':sharpe_vw})

    # --- Calculate High-Minus-Low (H-L) Portfolio ---
    zeronet_monthly = pd.DataFrame(); hl_calculated = False
    if 0 in monthly_agg_data and 9 in monthly_agg_data and not monthly_agg_data[0].empty and not monthly_agg_data[9].empty:
        long_monthly=monthly_agg_data[9].set_index('MonthYear'); short_monthly=monthly_agg_data[0].set_index('MonthYear'); common_index=long_monthly.index.intersection(short_monthly.index)
        if not common_index.empty:
             zeronet_monthly = long_monthly.loc[common_index].subtract(short_monthly.loc[common_index], fill_value=0).rename(columns=lambda x: x+'_HL'); # Subtract short from long
             if 'DecileRank_HL' in zeronet_monthly.columns: zeronet_monthly = zeronet_monthly.drop(columns=['DecileRank_HL'],errors='ignore')
             zeronet_monthly = zeronet_monthly.reset_index(); hl_calculated = True
             # Calculate H-L metrics
             ew_mean_ret_hl=zeronet_monthly["excess_return_portfolio_ew_HL"].mean(); vw_mean_ret_hl=zeronet_monthly["excess_return_portfolio_vw_HL"].mean(); ew_mean_pred_hl=zeronet_monthly["pred_excess_return_portfolio_ew_HL"].mean(); vw_mean_pred_hl=zeronet_monthly["pred_excess_return_portfolio_vw_HL"].mean()
             # Sharpe for H-L uses Std Dev of H-L RAW returns
             std_ew_raw_hl=zeronet_monthly["return_portfolio_ew_HL"].std(); std_vw_raw_hl=zeronet_monthly["return_portfolio_vw_HL"].std()
             sharpe_ew_hl=(ew_mean_ret_hl/std_ew_raw_hl)*np.sqrt(12) if std_ew_raw_hl>1e-9 else np.nan; sharpe_vw_hl=(vw_mean_ret_hl/std_vw_raw_hl)*np.sqrt(12) if std_vw_raw_hl>1e-9 else np.nan
             decile_results.append({'DecileRank':'H-L','ew_mean_pred':ew_mean_pred_hl,'ew_mean_ret':ew_mean_ret_hl,'std_ew_ret':std_ew_raw_hl,'sharpe_ew':sharpe_ew_hl,'vw_mean_pred':vw_mean_pred_hl,'vw_mean_ret':vw_mean_ret_hl,'std_vw_ret':std_vw_raw_hl,'sharpe_vw':sharpe_vw_hl})
        else: print("  ADVARSEL: Ingen overlappende måneder for H-L.")
    else: print("  ADVARSEL: Kan ikke beregne H-L (mangler data for desil 0 eller 9).");

    if not decile_results: print(f"  FEIL: Ingen desilresultater ({model_name_label})."); return (pd.DataFrame(),)*6
    results_summary_df = pd.DataFrame(decile_results).set_index('DecileRank')

    # --- Format Decile Output Tables (Table 7 style) ---
    def format_decile_table(summary_df, weight_scheme): # Use GLM+H table format (Pred, Avg, SD, SR)
        cols_map = {'ew_mean_pred':'Pred','ew_mean_ret':'Avg','std_ew_ret':'SD','sharpe_ew':'SR'} if weight_scheme=='EW' else {'vw_mean_pred':'Pred','vw_mean_ret':'Avg','std_vw_ret':'SD','sharpe_vw':'SR'}
        sub_df=summary_df[[k for k in cols_map.keys() if k in summary_df.columns]].rename(columns=cols_map)
        for col in ['Pred','Avg','SD']: # Convert relevant columns to percentage points for display
            if col in sub_df.columns: sub_df[col] = sub_df[col] * 100
        def map_index(x):
            if x==0: return 'Low (L)';
            if x==9: return 'High (H)';
            if x=='H-L': return 'H-L'
            try: return str(int(x)+1)
            except: return str(x)
        sub_df.index = sub_df.index.map(map_index); desired_order=['Low (L)','2','3','4','5','6','7','8','9','High (H)','H-L']; sub_df=sub_df.reindex([idx for idx in desired_order if idx in sub_df.index])
        return sub_df[[col for col in ['Pred','Avg','SD','SR'] if col in sub_df.columns]] # Ensure correct column order

    ew_table_unform = format_decile_table(results_summary_df, 'EW'); vw_table_unform = format_decile_table(results_summary_df, 'VW')
    ew_table = ew_table_unform.copy(); vw_table = vw_table_unform.copy()
    # Format numbers as strings with 2 decimal places
    for df_tbl in [ew_table, vw_table]:
        for col in ['Pred', 'Avg', 'SD']:
             if col in df_tbl.columns: df_tbl[col] = df_tbl[col].map('{:.2f}'.format).replace('nan','N/A')
        if 'SR' in df_tbl.columns: df_tbl['SR'] = df_tbl['SR'].map('{:.2f}'.format).replace('nan','N/A')
    print(f"\n--- Ytelsestabell ({model_name_label} Desiler) - EW ---"); print(ew_table); print(f"\n--- Ytelsestabell ({model_name_label} Desiler) - VW ---"); print(vw_table)

    # --- H-L Turnover, Drawdown, and Risk-Adjusted Performance (Table 8 style) ---
    print(f"\n--- Analyse av H-L Portefølje ({model_name_label}, Turnover, Drawdown, Risk-Adj. Perf.) ---")
    turnover_ew, turnover_vw, maxDD_ew, maxDD_vw, max_loss_ew, max_loss_vw = (np.nan,) * 6; hl_metrics_calculated = False
    if hl_calculated and not zeronet_monthly.empty and monthly_data_dict: # Check if H-L was calculated
        # Calculate Turnover for H-L
        all_months_weights = pd.concat(monthly_data_dict.values(), ignore_index=True)
        if 0 in all_months_weights['DecileRank'].unique() and 9 in all_months_weights['DecileRank'].unique():
             long_weights = all_months_weights[all_months_weights['DecileRank']==9][['MonthYear','Instrument','eq_weights','me_weights']]
             short_weights = all_months_weights[all_months_weights['DecileRank']==0][['MonthYear','Instrument','eq_weights','me_weights']]; short_weights[['eq_weights','me_weights']]*=-1 # Short positions have negative weight
             hl_weights = pd.concat([long_weights,short_weights]).sort_values(['Instrument','MonthYear'])
             # Calculate trade size for each instrument between consecutive months
             hl_weights['eq_weights_lead1'] = hl_weights.groupby('Instrument')['eq_weights'].shift(-1).fillna(0); hl_weights['me_weights_lead1'] = hl_weights.groupby('Instrument')['me_weights'].shift(-1).fillna(0)
             hl_weights['trade_ew'] = abs(hl_weights['eq_weights_lead1']-hl_weights['eq_weights']); hl_weights['trade_vw'] = abs(hl_weights['me_weights_lead1']-hl_weights['me_weights'])
             # Calculate monthly turnover (sum of absolute trades / 2) - exclude last month
             last_month_hl = hl_weights['MonthYear'].max(); monthly_turnover = hl_weights[hl_weights['MonthYear'] != last_month_hl].groupby('MonthYear').agg(sum_trade_ew=('trade_ew','sum'),sum_trade_vw=('trade_vw','sum'))
             if not monthly_turnover.empty: turnover_ew=monthly_turnover['sum_trade_ew'].mean()/2; turnover_vw=monthly_turnover['sum_trade_vw'].mean()/2; print(f"  Avg Turnover (H-L): EW={turnover_ew*100:.2f}%, VW={turnover_vw*100:.2f}%")
        else: print("  Kan ikke beregne turnover for H-L (mangler desil 0/9 i vektdata).")

        # Calculate Drawdowns and Max Loss using H-L EXCESS returns
        maxDD_ew=MDD(zeronet_monthly['excess_return_portfolio_ew_HL']); maxDD_vw=MDD(zeronet_monthly['excess_return_portfolio_vw_HL']); print(f"  Max Drawdown (H-L, Excess Ret): EW={abs(maxDD_ew)*100:.2f}%, VW={abs(maxDD_vw)*100:.2f}%")
        # Max 1M loss is the minimum monthly excess return
        max_loss_ew=zeronet_monthly['excess_return_portfolio_ew_HL'].min(); max_loss_vw=zeronet_monthly['excess_return_portfolio_vw_HL'].min(); print(f"  Max 1M Loss (H-L, Excess Ret): EW={max_loss_ew*100:.2f}%, VW={max_loss_vw*100:.2f}%")
        hl_metrics_calculated = True
    else: print("  Kan ikke utføre H-L turnover/drawdown analyse (mangler H-L månedsdata).")

    # --- Benchmark Comparison & Factor Regression Setup ---
    benchmark_name = "OSEBX" # Set your benchmark name
    print(f"\n--- Sammenligning med Benchmark ({benchmark_name}) og Faktor Modell ---")
    alpha_ew, t_alpha_ew, r2_reg_ew = np.nan, np.nan, np.nan; alpha_vw, t_alpha_vw, r2_reg_vw = np.nan, np.nan, np.nan
    factor_data_loaded = False; factors = None; bench_data_loaded = False; bench_zeronet = pd.DataFrame(); mean_ret_bench_pct, std_bench_raw_pct, sr_bench, mdd_bench_pct = (np.nan,) * 4

    # --- Load Factor Data (COMMENTED OUT - Requires FF5+Mom data file and statsmodels import) ---
    # import statsmodels.api as sm # Import if running regressions
    # if ff_factor_file and os.path.exists(ff_factor_file):
    #     try:
    #         print(f"  Laster faktordata fra: {ff_factor_file}")
    #         factors = pd.read_csv(ff_factor_file, parse_dates=['Date']) # Adjust Date column name if needed
    #         factors['MonthYear'] = factors['Date'].dt.to_period('M')
    #         factors = factors.drop_duplicates(subset=['MonthYear'], keep='last')
    #         # *** CHECK YOUR FACTOR COLUMN NAMES HERE ***
    #         factor_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   '] # Example names
    #         if all(c in factors.columns for c in factor_cols):
    #              # Ensure factors are decimals (e.g., 0.01, not 1.0) if necessary
    #              # if factors[factor_cols].abs().max().max() > 0.5: factors[factor_cols] /= 100
    #              factors = factors[['MonthYear'] + factor_cols]
    #              factor_data_loaded = True; print(f"  Faktordata lastet ({len(factors)} måneder).")
    #         else: print(f"  FEIL: Mangler påkrevde faktorkolonner i {ff_factor_file}. Funnet: {factors.columns.tolist()}"); factors = None
    #     except Exception as e: print(f"  FEIL ved lasting/prosessering av faktordata: {e}"); factors = None
    # else: print("  Info: Sti til FF+Mom faktordatafil ikke spesifisert eller fil ikke funnet. Hopper over faktorregresjon.")

    # --- Load Benchmark Data (COMMENTED OUT - Requires OSEBX data file) ---
    # if benchmark_file and os.path.exists(benchmark_file) and hl_calculated and not zeronet_monthly.empty: # Ensure H-L exists
    #     try:
    #         print(f"  Laster benchmark data ({benchmark_name}) fra: {benchmark_file}")
    #         bench = pd.read_csv(benchmark_file, parse_dates=['Date']) # Adjust column names if needed ('Date', 'Close')
    #         bench = bench.sort_values('Date'); bench['MonthYear'] = bench['Date'].dt.to_period('M'); bench = bench.drop_duplicates(subset=['MonthYear'], keep='last')
    #         bench['monthly_return_bench'] = bench['Close'].pct_change()
    #         # Merge with portfolio data to get consistent RiskFreeRate for excess return calc
    #         avg_monthly_rf = portfolio_data.groupby('MonthYear')['NextMonthRiskFreeRate_t+1'].mean().reset_index() # Use RF rate for t+1
    #         bench = pd.merge(bench[['MonthYear', 'monthly_return_bench']], avg_monthly_rf, on='MonthYear', how='left')
    #         bench['monthly_excess_return_bench'] = bench['monthly_return_bench'] - bench['NextMonthRiskFreeRate_t+1']
    #         # Merge benchmark returns with H-L returns
    #         bench_zeronet = pd.merge(bench[['MonthYear','monthly_return_bench','monthly_excess_return_bench']], zeronet_monthly, on='MonthYear', how='inner').dropna()
    #         if not bench_zeronet.empty:
    #             bench_data_loaded = True; print(f"  Benchmark data ({benchmark_name}) merget med H-L data ({len(bench_zeronet)} måneder).")
    #             # Calculate benchmark metrics on the overlapping period
    #             mean_ret_bench_pct = bench_zeronet["monthly_excess_return_bench"].mean() * 100
    #             std_bench_raw_pct = bench_zeronet["monthly_return_bench"].std() * 100 # Std of raw for Sharpe
    #             sr_bench = (bench_zeronet["monthly_excess_return_bench"].mean() / bench_zeronet["monthly_return_bench"].std()) * np.sqrt(12) if bench_zeronet["monthly_return_bench"].std() > 1e-9 else np.nan
    #             mdd_bench_pct = abs(MDD(bench_zeronet["monthly_excess_return_bench"])) * 100
    #         else: print("  ADVARSEL: Ingen overlappende måneder funnet mellom H-L portefølje og benchmark data.")
    #     except Exception as e: print(f"  FEIL under lasting eller prosessering av benchmark data: {e}")
    # else:
    #      if not benchmark_file or not os.path.exists(benchmark_file): print(f"  Info: Benchmark datafil '{benchmark_file}' ikke funnet.")
    #      if not hl_calculated or zeronet_monthly.empty: print("  Info: H-L portefølje ikke tilgjengelig for benchmark sammenligning.")
    #      print("  Hopper over benchmark sammenligning.")

    # --- Factor Regression for H-L Portfolio (COMMENTED OUT - Requires 'factors' and 'bench_zeronet') ---
    # if factor_data_loaded and bench_data_loaded and not bench_zeronet.empty: # Need factors and H-L returns on same timeframe
    #     print("  Kjører FF5+Mom faktorregresjon for H-L portefølje...")
    #     analysis_data_reg = pd.merge(bench_zeronet, factors, on='MonthYear', how='inner') # Merge H-L/Bench data with factors
    #     if not analysis_data_reg.empty:
    #         X_factors = sm.add_constant(analysis_data_reg[factor_cols]) # Prepare factor matrix with intercept
    #         for weight_scheme in ['ew', 'vw']:
    #             hl_ret_col = f'excess_return_portfolio_{weight_scheme}_HL'
    #             y_hl = analysis_data_reg[hl_ret_col]
    #             try:
    #                 model_reg = sm.OLS(y_hl, X_factors, missing='drop').fit()
    #                 alpha = model_reg.params['const'] * 100 # Monthly alpha in percent
    #                 t_alpha = model_reg.tvalues['const']
    #                 r2_reg = model_reg.rsquared_adj
    #                 print(f"    {weight_scheme.upper()} H-L FF5+Mom Alpha: {alpha:.3f}% (t={t_alpha:.2f}), Adj R2: {r2_reg:.3f}")
    #                 if weight_scheme == 'ew': alpha_ew, t_alpha_ew, r2_reg_ew = alpha, t_alpha, r2_reg
    #                 else: alpha_vw, t_alpha_vw, r2_reg_vw = alpha, t_alpha, r2_reg
    #             except Exception as e_reg: print(f"    FEIL under OLS regresjon for H-L {weight_scheme}: {e_reg}")
    #     else: print("    Ingen overlappende data for H-L faktorregresjon.")
    # else:
    #     if not factor_data_loaded: print("    Faktordata ikke lastet, hopper over H-L regresjon.")
    #     if not bench_data_loaded or bench_zeronet.empty: print("    H-L/Benchmark data ikke klar, hopper over H-L regresjon.")

    # --- Create Risk-Adjusted Performance Summary Tables (H-L) ---
    if hl_metrics_calculated: # Only create tables if H-L metrics exist
        hl_res = results_summary_df.loc['H-L'] if 'H-L' in results_summary_df.index else pd.Series(dtype=float)
        # Define rows for the risk table (Table 8 style)
        index_perf_hl = ["Mean Excess Return [%]", 'Std Dev (Raw) [%]', "Ann. Sharpe Ratio", "Max Drawdown [%]", "Avg Monthly Turnover [%]", "FF5+Mom Alpha [%]", "t(Alpha)", "FF5+Mom Adj R2"]
        # Populate H-L data
        ew_chart_hl_data = {f'{model_name_label} H-L': [hl_res.get('ew_mean_ret', np.nan) * 100, hl_res.get('std_ew_ret', np.nan) * 100, hl_res.get('sharpe_ew', np.nan), abs(maxDD_ew) * 100 if pd.notna(maxDD_ew) else np.nan, turnover_ew * 100 if pd.notna(turnover_ew) else np.nan, alpha_ew, t_alpha_ew, r2_reg_ew]}
        vw_chart_hl_data = {f'{model_name_label} H-L': [hl_res.get('vw_mean_ret', np.nan) * 100, hl_res.get('std_vw_ret', np.nan) * 100, hl_res.get('sharpe_vw', np.nan), abs(maxDD_vw) * 100 if pd.notna(maxDD_vw) else np.nan, turnover_vw * 100 if pd.notna(turnover_vw) else np.nan, alpha_vw, t_alpha_vw, r2_reg_vw]}
        # Add Benchmark column if data was loaded
        if bench_data_loaded:
            bench_col_data = [mean_ret_bench_pct, std_bench_raw_pct, sr_bench, mdd_bench_pct, 0, np.nan, np.nan, np.nan] # Benchmark has 0 turnover, no factor alpha by definition here
            ew_chart_hl_data[benchmark_name] = bench_col_data
            vw_chart_hl_data[benchmark_name] = bench_col_data
            # Ensure benchmark is the first column
            ew_chart_hl = pd.DataFrame(ew_chart_hl_data, index=index_perf_hl)[[benchmark_name, f'{model_name_label} H-L']]
            vw_chart_hl = pd.DataFrame(vw_chart_hl_data, index=index_perf_hl)[[benchmark_name, f'{model_name_label} H-L']]
        else: # Create tables without benchmark column
            ew_chart_hl = pd.DataFrame(ew_chart_hl_data, index=index_perf_hl)
            vw_chart_hl = pd.DataFrame(vw_chart_hl_data, index=index_perf_hl)

        print(f"\n--- Risk-Adjusted Performance ({model_name_label} H-L vs {benchmark_name}, EW) ---"); print(ew_chart_hl.round(3))
        print(f"\n--- Risk-Adjusted Performance ({model_name_label} H-L vs {benchmark_name}, VW) ---"); print(vw_chart_hl.round(3))

    # --- Long-Only (Top Decile) Portfolio Analysis ---
    print(f"\n--- Analyse av Long-Only ({model_name_label}, Topp Desil) Portefølje ---")
    rank_9_df = decile_portfolios.get(9, pd.DataFrame()) # Get data for decile 9
    turnover_ew_long, turnover_vw_long, maxDD_ew_long, maxDD_vw_long, max_loss_ew_long, max_loss_vw_long = (np.nan,) * 6
    long_metrics_calculated = False; alpha_long_ew, t_alpha_long_ew, r2_reg_long_ew = np.nan, np.nan, np.nan; alpha_long_vw, t_alpha_long_vw, r2_reg_long_vw = np.nan, np.nan, np.nan
    if not rank_9_df.empty:
        # Turnover for Long-Only
        long_weights = rank_9_df[['MonthYear','Instrument','eq_weights','me_weights']].copy().sort_values(['Instrument','MonthYear'])
        long_weights['eq_weights_lead1'] = long_weights.groupby('Instrument')['eq_weights'].shift(-1).fillna(0); long_weights['me_weights_lead1'] = long_weights.groupby('Instrument')['me_weights'].shift(-1).fillna(0)
        long_weights['trade_ew'] = abs(long_weights['eq_weights_lead1']-long_weights['eq_weights']); long_weights['trade_vw'] = abs(long_weights['me_weights_lead1']-long_weights['me_weights'])
        last_month_long = long_weights['MonthYear'].max(); monthly_turnover_long=long_weights[long_weights['MonthYear']!=last_month_long].groupby('MonthYear').agg(sum_trade_ew=('trade_ew','sum'),sum_trade_vw=('trade_vw','sum'))
        if not monthly_turnover_long.empty: turnover_ew_long=monthly_turnover_long['sum_trade_ew'].mean()/2; turnover_vw_long=monthly_turnover_long['sum_trade_vw'].mean()/2; print(f"  Avg Turnover (Long Only): EW={turnover_ew_long*100:.2f}%, VW={turnover_vw_long*100:.2f}%")
        else: print("  Kan ikke beregne turnover for Long-Only.")

        # Drawdown and Max Loss for Long-Only using aggregated monthly returns
        long_monthly_agg = monthly_agg_data.get(9, pd.DataFrame())
        if not long_monthly_agg.empty:
            maxDD_ew_long=MDD(long_monthly_agg["excess_return_portfolio_ew"]); maxDD_vw_long=MDD(long_monthly_agg["excess_return_portfolio_vw"]); print(f"  Max Drawdown (Long Only, Excess Ret): EW={abs(maxDD_ew_long)*100:.2f}%, VW={abs(maxDD_vw_long)*100:.2f}%")
            max_loss_ew_long=long_monthly_agg["excess_return_portfolio_ew"].min(); max_loss_vw_long=long_monthly_agg["excess_return_portfolio_vw"].min(); print(f"  Max 1M Loss (Long Only, Excess Ret): EW={max_loss_ew_long*100:.2f}%, VW={max_loss_vw_long*100:.2f}%")
            long_metrics_calculated = True

            # --- Factor Regression for Long-Only Portfolio (COMMENTED OUT) ---
            # if factor_data_loaded and not long_monthly_agg.empty:
            #     print("  Kjører FF5+Mom faktorregresjon for Long-Only portefølje...")
            #     analysis_data_long_reg = pd.merge(long_monthly_agg, factors, on='MonthYear', how='inner')
            #     if not analysis_data_long_reg.empty:
            #         X_factors_long = sm.add_constant(analysis_data_long_reg[factor_cols])
            #         for weight_scheme in ['ew', 'vw']:
            #              long_ret_col = f'excess_return_portfolio_{weight_scheme}' # Use excess return as dependent var
            #              y_long = analysis_data_long_reg[long_ret_col]
            #              try:
            #                  model_long_reg = sm.OLS(y_long, X_factors_long, missing='drop').fit()
            #                  alpha_long = model_long_reg.params['const'] * 100 # Monthly alpha %
            #                  t_alpha_long = model_long_reg.tvalues['const']
            #                  r2_reg_long = model_long_reg.rsquared_adj
            #                  print(f"    {weight_scheme.upper()} Long FF5+Mom Alpha: {alpha_long:.3f}% (t={t_alpha_long:.2f}), Adj R2: {r2_reg_long:.3f}")
            #                  if weight_scheme == 'ew': alpha_long_ew, t_alpha_long_ew, r2_reg_long_ew = alpha_long, t_alpha_long, r2_reg_long
            #                  else: alpha_long_vw, t_alpha_long_vw, r2_reg_long_vw = alpha_long, t_alpha_long, r2_reg_long
            #              except Exception as e_reg_long: print(f"    FEIL under OLS regresjon for Long {weight_scheme}: {e_reg_long}")
            #     else: print("    Ingen overlappende data for Long-Only faktorregresjon.")
            # else:
            #      if not factor_data_loaded: print("    Faktordata ikke lastet, hopper over Long-Only regresjon.")

            # --- Create Risk-Adjusted Performance Summary Tables for Long-Only ---
            if long_metrics_calculated:
                long_res = results_summary_df.loc[9] if 9 in results_summary_df.index else pd.Series(dtype=float)
                index_perf_long = ["Mean Excess Return [%]", 'Std Dev (Raw) [%]', "Ann. Sharpe Ratio", "Max Drawdown [%]", "Avg Monthly Turnover [%]", "FF5+Mom Alpha [%]", "t(Alpha)", "FF5+Mom Adj R2"]
                ew_chart_long_data = {f'{model_name_label} Long': [long_res.get('ew_mean_ret', np.nan) * 100, long_res.get('std_ew_ret', np.nan) * 100, long_res.get('sharpe_ew', np.nan), abs(maxDD_ew_long) * 100 if pd.notna(maxDD_ew_long) else np.nan, turnover_ew_long * 100 if pd.notna(turnover_ew_long) else np.nan, alpha_long_ew, t_alpha_long_ew, r2_reg_long_ew]}
                vw_chart_long_data = {f'{model_name_label} Long': [long_res.get('vw_mean_ret', np.nan) * 100, long_res.get('std_vw_ret', np.nan) * 100, long_res.get('sharpe_vw', np.nan), abs(maxDD_vw_long) * 100 if pd.notna(maxDD_vw_long) else np.nan, turnover_vw_long * 100 if pd.notna(turnover_vw_long) else np.nan, alpha_long_vw, t_alpha_long_vw, r2_reg_long_vw]}
                if bench_data_loaded:
                    bench_col_data_long = [mean_ret_bench_pct, std_bench_raw_pct, sr_bench, mdd_bench_pct, 0, np.nan, np.nan, np.nan]
                    ew_chart_long_data[benchmark_name] = bench_col_data_long; vw_chart_long_data[benchmark_name] = bench_col_data_long
                    ew_chart_long = pd.DataFrame(ew_chart_long_data, index=index_perf_long)[[benchmark_name, f'{model_name_label} Long']]
                    vw_chart_long = pd.DataFrame(vw_chart_long_data, index=index_perf_long)[[benchmark_name, f'{model_name_label} Long']]
                else:
                    ew_chart_long = pd.DataFrame(ew_chart_long_data, index=index_perf_long); vw_chart_long = pd.DataFrame(vw_chart_long_data, index=index_perf_long)
                print(f"\n--- Risk-Adjusted Performance ({model_name_label} Long Only vs {benchmark_name}, EW) ---"); print(ew_chart_long.round(3)); print(f"\n--- Risk-Adjusted Performance ({model_name_label} Long Only vs {benchmark_name}, VW) ---"); print(vw_chart_long.round(3))
        else: print("  Ingen aggregerte månedsdata for Long-Only (Decile 9). Kan ikke beregne Drawdown/Max Loss.")
    else: print(f"  Ingen data for Long-Only porteføljeanalyse ({model_name_label}, Decile 9).")

    # --- Cumulative Return Plots (Figure 9 style) ---
    print(f"\n--- Genererer kumulative avkastningsplott ({model_name_label}, Excess Returns) ---")
    plot_data = {}; # Prepare data for plots
    # Get monthly excess returns for Deciles 0 (Short) and 9 (Long)
    for j in [0, 9]:
        if j in monthly_agg_data and not monthly_agg_data[j].empty:
            df_agg=monthly_agg_data[j].set_index('MonthYear').sort_index();
            if not df_agg.empty:
                # Calculate cumulative product of (1 + excess return)
                df_agg[f'cum_ret_ew_{j}']=(1+df_agg['excess_return_portfolio_ew']).cumprod()-1;
                df_agg[f'cum_ret_vw_{j}']=(1+df_agg['excess_return_portfolio_vw']).cumprod()-1;
                plot_data[j]=df_agg
    # Add benchmark cumulative return if available
    # if bench_data_loaded and not bench_zeronet.empty:
    #     bench_plot=bench_zeronet.set_index('MonthYear').sort_index(); bench_plot['cum_ret_bench']=(1+bench_plot['monthly_excess_return_bench']).cumprod()-1; plot_data['benchmark']=bench_plot

    def plot_cumulative_returns(plot_data_dict, weight_scheme, model_label, benchmark_label):
        fig, ax = plt.subplots(figsize=(15, 7)); legend_items=[]
        # Plot Long (Decile 9)
        if 9 in plot_data_dict: col_name=f'cum_ret_{weight_scheme.lower()}_9'; plot_data_dict[9][col_name].plot(ax=ax, label=f'{model_label} Long {weight_scheme.upper()}'); legend_items.append(f'{model_label} Long {weight_scheme.upper()}')
        # Plot Short (Decile 0)
        if 0 in plot_data_dict: col_name=f'cum_ret_{weight_scheme.lower()}_0'; plot_data_dict[0][col_name].plot(ax=ax, label=f'{model_label} Short {weight_scheme.upper()}', linestyle=':'); legend_items.append(f'{model_label} Short {weight_scheme.upper()}')
        # Plot Benchmark (Optional)
        # if 'benchmark' in plot_data_dict: plot_data_dict['benchmark']['cum_ret_bench'].plot(ax=ax, label=benchmark_label, linestyle='--', color='grey'); legend_items.append(benchmark_label) # Uncomment to plot benchmark
        if legend_items: ax.set_title(f'Kumulativ Excess Avkastning ({model_label} {weight_scheme.upper()} Vektet)'); ax.set_ylabel('Kumulativ Excess Avkastning'); ax.set_xlabel('Måned'); ax.legend(legend_items); ax.grid(True); fig.tight_layout(); plt.show()
        else: plt.close(fig); print(f"Ingen {weight_scheme.upper()} plottdata tilgjengelig for {model_label}.")

    plot_cumulative_returns(plot_data, 'EW', model_name_label, benchmark_name)
    plot_cumulative_returns(plot_data, 'VW', model_name_label, benchmark_name)

    print(f"--- Detaljert Porteføljeanalyse Fullført ({model_name_label}) ---")
    # Return the generated tables
    return ew_table, vw_table, ew_chart_hl, vw_chart_hl, ew_chart_long, vw_chart_long


# -----------------------------------------------------------------------------
# Step 6.7: Variable Importance Function (MODIFIED for NN - run ONCE after loop)
# -----------------------------------------------------------------------------
def calculate_variable_importance(df_standardized, features, target_col, train_idx, val_idx,
                                  nn_config, # Pass NN architecture
                                  optimal_lambda1, # Pass optimal lambda from LAST window
                                  optimal_lr,      # Pass optimal LR from LAST window
                                  epochs=100, batch_size=10000, ensemble_size=10):
    """
    Calculates permutation importance for NN using data from the LAST provided indices
    and the optimal hyperparameters found in the LAST window's tuning.
    Retrains the ensemble for each permuted feature (computationally expensive).
    Uses the zeroing-out method for permutation.
    """
    model_name = nn_config['name']
    print(f"\n--- Starter Variabel Viktighet Analyse ({model_name} Ensemble, Siste Vindu Data) ---")
    print(f"    Bruker optimale hyperparametre fra siste vindu: L1={optimal_lambda1:.6f}, LR={optimal_lr:.6f}")
    print("    *** ADVARSEL: Denne prosessen (ensemble retraining per feature) er SVÆRT tidkrevende! ***")

    if train_idx is None or val_idx is None or train_idx.empty or val_idx.empty: print("  FEIL: Ugyldige VI indekser."); return pd.DataFrame()
    if pd.isna(optimal_lambda1) or pd.isna(optimal_lr): print("  FEIL: Mangler optimale hyperparametre for VI."); return pd.DataFrame()

    print(f"  Bruker siste Train index ({len(train_idx)} obs), siste Val index ({len(val_idx)} obs)")
    # Prepare data from the last train/val window
    X_train = df_standardized.loc[train_idx, features].values; y_train = df_standardized.loc[train_idx, target_col].values
    X_val = df_standardized.loc[val_idx, features].values; y_val = df_standardized.loc[val_idx, target_col].values
    X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))

    # Check data validity
    valid_y_mask = np.isfinite(y_train_val)
    if X_train_val.shape[0] < 2 or np.nanstd(y_train_val[valid_y_mask]) < 1e-15: print("  FEIL: VI data/varians."); return pd.DataFrame()
    ss_tot = np.sum((y_train_val[valid_y_mask] - y_train_val[valid_y_mask].mean())**2)
    if ss_tot < 1e-15: print("  ADVARSEL: VI SS_tot er nesten null."); return pd.DataFrame()

    input_shape = X_train.shape[1]
    terminate_nan_vi = callbacks.TerminateOnNaN()

    # --- Train Base Ensemble Model for VI (using last window's optimal params on last Train+Val) ---
    print(f"  Trener base {model_name} ensemble for VI (på siste Train+Val)...")
    base_preds_ensemble = []
    base_r2 = np.nan
    try:
        for ens_idx in range(ensemble_size):
            K.clear_session(); tf.random.set_seed(SEED + ens_idx + 3*ensemble_size) # Use different seeds
            base_model_vi = build_nn_model(input_shape, nn_config, optimal_lambda1)
            optimizer_base_vi = Adam(learning_rate=optimal_lr)
            base_model_vi.compile(optimizer=optimizer_base_vi, loss='mean_squared_error')
            # Train on the combined Train+Val set from the last window
            hist_base = base_model_vi.fit(X_train_val, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan_vi], verbose=0)
            if not np.isnan(hist_base.history['loss']).any():
                # Predict on the same Train+Val set to get baseline performance
                base_preds_ensemble.append(base_model_vi.predict(X_train_val, batch_size=batch_size).flatten())
            else:
                print(f"      WARN ({model_name}): NaN loss i base VI ensemble {ens_idx}. Stopper base trening.")
                base_preds_ensemble = []; break # Fail fast if base model fails

        if not base_preds_ensemble: raise ValueError(f"Base model VI ensemble training failed ({model_name}).")

        # Calculate baseline R2 on the Train+Val data
        base_preds_avg = np.mean(np.array(base_preds_ensemble), axis=0)
        valid_base_pred_mask = np.isfinite(base_preds_avg)
        combined_valid_mask = valid_y_mask & valid_base_pred_mask
        ss_res_base = np.sum((y_train_val[combined_valid_mask] - base_preds_avg[combined_valid_mask])**2)
        # Use the pre-calculated ss_tot based on valid y values
        base_r2 = 1 - (ss_res_base / ss_tot)
        print(f"  Base R2 ({model_name} ensemble, siste Train+Val): {base_r2:.4f}")

    except Exception as e_base:
        print(f"  FEIL under trening av base VI modell ({model_name}): {e_base}"); return pd.DataFrame()
    if np.isnan(base_r2): print(f"  FEIL: Base R2 for VI er NaN."); return pd.DataFrame()


    # --- Permutation Importance (Zeroing out & Retraining Ensemble) ---
    print(f"  Beregner {model_name} viktighet (zeroing out & retraining ensemble)...")
    importance_results = {}
    vi_start_time = time.time()
    for feature_idx, feature_name in enumerate(features):
        print(f"    Permuterer feature: {feature_name} ({feature_idx+1}/{len(features)})...", end="")
        feat_time_start = time.time()
        # Create permuted dataset by zeroing out the feature
        X_train_val_permuted = X_train_val.copy(); X_train_val_permuted[:, feature_idx] = 0
        permuted_preds_ensemble = []
        permuted_r2 = np.nan
        try:
            # Retrain the FULL ensemble with the permuted data
            for ens_idx in range(ensemble_size):
                K.clear_session(); tf.random.set_seed(SEED + ens_idx + (feature_idx+4)*ensemble_size) # Unique seeds per feature/ensemble
                permuted_model = build_nn_model(input_shape, nn_config, optimal_lambda1)
                optimizer_perm = Adam(learning_rate=optimal_lr)
                permuted_model.compile(optimizer=optimizer_perm, loss='mean_squared_error')
                # Train on the permuted Train+Val data
                hist_perm = permuted_model.fit(X_train_val_permuted, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan_vi], verbose=0)
                if not np.isnan(hist_perm.history['loss']).any():
                    # Predict on the same permuted Train+Val data
                    permuted_preds_ensemble.append(permuted_model.predict(X_train_val_permuted, batch_size=batch_size).flatten())
                else:
                    # print(f" -> WARN: NaN loss i VI ensemble for {feature_name}...") # Verbose
                    permuted_preds_ensemble = []; break # Fail fast for this feature

            if not permuted_preds_ensemble:
                 # If ensemble failed, importance is effectively 0 (R2 reduction is 0 or negative)
                 print(f" -> WARN: Ensemble retraining failed. Assigning R2 = base R2.")
                 permuted_r2 = base_r2
            else:
                # Calculate R2 for the permuted feature
                permuted_preds_avg = np.mean(np.array(permuted_preds_ensemble), axis=0)
                valid_perm_pred_mask = np.isfinite(permuted_preds_avg)
                combined_valid_mask_perm = valid_y_mask & valid_perm_pred_mask
                ss_res_permuted = np.sum((y_train_val[combined_valid_mask_perm] - permuted_preds_avg[combined_valid_mask_perm])**2)
                permuted_r2 = 1 - (ss_res_permuted / ss_tot)

        except Exception as e_perm:
            print(f" -> FEIL: {e_perm}. Assigning R2 = base R2.")
            permuted_r2 = base_r2 # Assign base R2 on error

        # Importance is the reduction in R2 compared to the baseline
        r2_reduction = base_r2 - permuted_r2
        importance_results[feature_name] = max(0, r2_reduction) if pd.notna(r2_reduction) else 0.0 # Ensure non-negative importance
        print(f" [R2 perm: {permuted_r2:.4f}, Red: {importance_results[feature_name]:.4f}, Time: {time.time() - feat_time_start:.1f}s]")

    total_vi_time = time.time() - vi_start_time
    print(f"  Total VI beregningstid: {total_vi_time:.1f}s")

    if not importance_results: print("  Ingen VI resultater."); return pd.DataFrame()
    imp_df = pd.DataFrame(importance_results.items(), columns=['Feature', 'R2_reduction'])
    total_reduction = imp_df['R2_reduction'].sum()
    # Normalize importance scores
    imp_df['Importance'] = imp_df['R2_reduction']/total_reduction if total_reduction > 1e-9 else 0.0
    imp_df = imp_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    print(f"\n--- Variabel Viktighet ({model_name} Ensemble, Top 10) ---"); print(imp_df[['Feature', 'Importance']].head(10).round(4))
    # Plotting
    plt.figure(figsize=(12, 8)); top_n = 20
    plot_df = imp_df.head(top_n).sort_values(by='Importance', ascending=True)
    plt.barh(plot_df['Feature'], plot_df['Importance']); plt.xlabel("Relativ Viktighet (Permutation - Ensemble Retrain)"); plt.ylabel("Feature"); plt.title(f'{model_name} Variabel Viktighet (Top {top_n}) - Siste Vindus Data'); plt.tight_layout(); plt.show()
    return imp_df[['Feature', 'Importance']]

# ------------------------------------------------
# Step 7: Analyze Prespecified Portfolios (Placeholder - Copied from GLM+H)
# ------------------------------------------------
def analyze_prespecified_portfolios(results_df, original_df_subset, portfolio_definitions_file=None, model_name_label="NN"):
    """ Placeholder function for analyzing prespecified portfolios. Needs implementation. """
    print(f"\n--- Starter Analyse av Prespesifiserte Porteføljer ({model_name_label}, Placeholder) ---")
    pred_col = f'yhat_{model_name_label.lower().replace("-","_").replace("+","h")}'
    portfolio_r2_results = pd.DataFrame()
    market_timing_results = pd.DataFrame()

    if portfolio_definitions_file is None or not os.path.exists(portfolio_definitions_file):
        print("  ADVARSEL: Fil med porteføljedefinisjoner mangler. Hopper over prespesifisert porteføljeanalyse.")
        print("            Forventer CSV med 'Date'/'MonthYear', 'Instrument', 'PortfolioName', 'Weight'")
        return portfolio_r2_results, market_timing_results

    # --- IMPLEMENTATION NEEDED ---
    # 1. Load portfolio weights from portfolio_definitions_file (e.g., CSV).
    #    Make sure dates/identifiers match.
    # 2. Merge weights with results_df (which has 'Date', 'Instrument', pred_col)
    #    and original_df_subset (which has 'Date', 'Instrument', 'NextMonthlyReturn_t+1').
    # 3. For each unique PortfolioName and MonthYear:
    #    a. Calculate the predicted portfolio return: sum(weight * yhat_t+1)
    #    b. Calculate the actual portfolio return: sum(weight * ret_t+1)
    # 4. Calculate the overall OOS R2 for each PortfolioName time series (like Table 5).
    #    Use the actual portfolio returns vs predicted portfolio returns.
    # 5. Calculate Market Timing metrics if desired (like Table 6 - requires more specific methodology).
    # --- END OF IMPLEMENTATION NEEDED ---

    print(f"--- Analyse av Prespesifiserte Porteføljer Fullført ({model_name_label}, Placeholder) ---")
    return portfolio_r2_results, market_timing_results


# --------------------------------------------------------------------------
# Step 8: Main Orchestration Function (Adapted for NNs with Yearly Refitting)
# --------------------------------------------------------------------------
# --- Define Yearly Split Parameters Here (Match GLM+H Defaults or Adjust) ---
INITIAL_TRAIN_YEARS_DEFAULT = 9   # Example: 1995-2003
VALIDATION_YEARS_DEFAULT = 6    # Example: 2004-2009
TEST_YEARS_PER_WINDOW_DEFAULT = 1 # Test 2010, 2011, ...

def run_analysis_for_subset(file_path, data_subset='all',
                            benchmark_file=None, ff_factor_file=None, portfolio_defs_file=None,
                            filter_portfolio_construction=False,
                            top_n=1000, bottom_n=1000, # For big/small subsets
                            nn_config=None, # Pass NN architecture
                            param_grid_nn=None, # Pass NN hyperparameter grid
                            epochs=100, batch_size=10000, patience=5, ensemble_size=10,
                            initial_train_years=INITIAL_TRAIN_YEARS_DEFAULT,
                            val_years=VALIDATION_YEARS_DEFAULT,
                            test_years=TEST_YEARS_PER_WINDOW_DEFAULT
                            ):
    """ Runs the full pipeline for one data subset using a specific NN config with YEARLY refitting. """
    if nn_config is None or param_grid_nn is None: raise ValueError("nn_config and param_grid_nn must be provided.")

    model_name = nn_config['name'] # e.g., "NN1"
    run_label = data_subset.capitalize()
    start_time = datetime.datetime.now()
    print(f"\n{'='*20} Starter Kjøring: {model_name} for '{run_label}' Firms (ÅRLIG Refitting) {'='*20}")
    if data_subset == 'big': print(f"(Definert som Topp {top_n} basert på MarketCap per måned)")
    if data_subset == 'small': print(f"(Definert som Bunn {bottom_n} basert på MarketCap per måned)")
    print(f"Starttid: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    # --- Load & Subset (Identical to GLM+H version) ---
    print("\n--- Steg 1: Laster og Forbereder Rådata ---")
    df_raw = load_prepare_data(file_path)
    if df_raw is None: return np.nan, None, None, (None,)*6, (None, None)
    if 'MarketCap_orig' not in df_raw.columns: print("FEIL: MarketCap_orig mangler."); return np.nan, None, None, (None,)*6, (None, None)
    if 'Date' not in df_raw.columns: print("FEIL: Date mangler."); return np.nan, None, None, (None,)*6, (None, None)
    print(f"\n--- Steg 1.1: Lager Subset: {run_label} ---")
    df = pd.DataFrame(); df_raw_mc = df_raw.dropna(subset=['MarketCap_orig', 'Date'])
    if data_subset == 'all': df = df_raw.copy()
    elif data_subset == 'big': df_raw_mc['MonthYear'] = df_raw_mc['Date'].dt.to_period('M'); df = df_raw_mc.groupby('MonthYear', group_keys=False).apply(lambda x: x.nlargest(top_n, "MarketCap_orig")); df = df.drop(columns=['MonthYear'])
    elif data_subset == 'small': df_raw_mc['MonthYear'] = df_raw_mc['Date'].dt.to_period('M'); df = df_raw_mc.groupby('MonthYear', group_keys=False).apply(lambda x: x.nsmallest(bottom_n, "MarketCap_orig")); df = df.drop(columns=['MonthYear'])
    else: print(f"FEIL: Ukjent subset '{data_subset}'."); return np.nan, None, None, (None,)*6, (None, None)
    if df.empty: print(f"FEIL: Subset '{run_label}' er tomt."); return np.nan, None, None, (None,)*6, (None, None)
    print(f"Subset '{run_label}' initiell form: {df.shape}")

    # --- Define Features, Standardize & Clean (Identical to GLM+H version) ---
    print("\n--- Steg 2 & 1.5: Definerer Features og Rank Standardiserer ---")
    nn_features = define_features(df)
    if not nn_features: print(f"FEIL: Ingen features funnet i subset '{run_label}'."); return np.nan, None, None, (None,)*6, (None, None)
    df = rank_standardize_features(df, nn_features)
    print("\n--- Steg 3: Renser Data (Missing/Inf/Filters) ---")
    essential_cols = ['TargetReturn_t', 'NextMonthlyReturn_t+1', 'MarketCap_orig', 'Date', 'Instrument']
    df = clean_data(df, nn_features, essential_cols, target="TargetReturn_t")
    if df.empty: print(f"FEIL: DataFrame er tom etter rensing for subset '{run_label}'."); return np.nan, None, None, (None,)*6, (None, None)
    nn_features = [f for f in nn_features if f in df.columns and df[f].nunique() > 1 and df[f].std() > 1e-9] # Refresh features
    if not nn_features: print("FEIL: Ingen features igjen etter datarensing."); return np.nan, None, None, (None,)*6, (None, None)
    df = df.sort_values(["Date", "Instrument"]).reset_index(drop=True)

    # --- Yearly Rolling Window Setup ---
    print(f"\n--- Steg 4: Setter opp ÅRLIG Rullerende Vindu (InitTrain={initial_train_years}, Val={val_years}, Test={test_years}) ---")
    results_list = []; model_metrics = defaultdict(lambda: defaultdict(list));
    yhat_col_name = f'yhat_{model_name.lower().replace("-", "_").replace("+","h")}' # e.g., yhat_nn1
    last_train_idx, last_val_idx = None, None # Store indices from the last window for VI
    last_optim_lambda1, last_optim_lr = np.nan, np.nan # Store params from last window for VI

    try:
        splits_generator = get_yearly_rolling_splits(df, initial_train_years, val_years, test_years)
        splits = list(splits_generator); num_windows = len(splits)
    except ValueError as e:
        print(f"FEIL ved generering av årlige rullerende vinduer: {e}\nAvslutter kjøring for {run_label}.")
        if 'Year' in df.columns: df.drop(columns=['Year'], inplace=True, errors='ignore')
        return np.nan, None, model_metrics, (None,)*6, (None, None)
    print(f"Antall årlige rullerende vinduer som skal kjøres: {num_windows}\n");
    if num_windows == 0: print("Ingen årlige vinduer å kjøre. Avslutter."); return np.nan, None, model_metrics, (None,)*6, (None, None)

    # --- Yearly Rolling Window Loop ---
    print(f"--- Starter {model_name} ÅRLIG Rullerende Vindu Trening & Prediksjon ---")
    for window, (train_idx, val_idx, test_idx, train_dates, val_dates, test_dates) in enumerate(splits):
        window_start_time = datetime.datetime.now(); window_num = window + 1; print("-" * 60) # Separator
        if test_idx.empty or val_idx.empty or train_idx.empty:
             print(f"Vindu {window_num} ({model_name}): Tomt train/val/test sett. Hopper over.")
             model_metrics[model_name]['oos_r2'].append(np.nan); model_metrics[model_name]['optim_lambda1'].append(np.nan); model_metrics[model_name]['optim_lr'].append(np.nan); model_metrics[model_name]['is_r2_train_val'].append(np.nan)
             continue

        X_train = df.loc[train_idx, nn_features].values; y_train = df.loc[train_idx, "TargetReturn_t"].values
        X_val = df.loc[val_idx, nn_features].values; y_val = df.loc[val_idx, "TargetReturn_t"].values
        X_test = df.loc[test_idx, nn_features].values; y_test = df.loc[test_idx, "TargetReturn_t"].values

        # Basic data checks for the window
        if np.isnan(y_train).all() or np.isnan(y_val).all() or X_train.shape[0]<2 or X_val.shape[0]<2 or np.nanstd(y_train) < 1e-9:
            print(f"Vindu {window_num} ({model_name}): Utilstrekkelig data/varians i train/val. Hopper over.")
            model_metrics[model_name]['oos_r2'].append(np.nan); model_metrics[model_name]['optim_lambda1'].append(np.nan); model_metrics[model_name]['optim_lr'].append(np.nan); model_metrics[model_name]['is_r2_train_val'].append(np.nan)
            continue

        # Check batch size vs train size
        current_batch_size = min(batch_size, X_train.shape[0])
        if current_batch_size < batch_size and X_train.shape[0] > 0: print(f"  INFO Vindu {window_num} ({model_name}): Batch size ({batch_size}) > train size ({X_train.shape[0]}). Bruker batch size = {current_batch_size}")
        elif X_train.shape[0] == 0: current_batch_size = 1 # Avoid batch size 0

        # *** Call the NN window function ***
        preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, optim_lambda1, optim_lr = run_nn_on_window(
            X_train, y_train, X_val, y_val, X_test, y_test,
            nn_config=nn_config,
            param_grid=list(ParameterGrid(param_grid_nn)), # Pass the grid combinations
            epochs=epochs,
            batch_size=current_batch_size, # Use potentially adjusted batch size
            patience=patience,
            ensemble_size=ensemble_size
        )

        # --- Store Results & Last Window Info for VI ---
        if test_idx.shape[0] > 0 and preds_oos is not None:
             # Create df for OOS results of this window
             window_predictions = {'Date': df.loc[test_idx, 'Date'].values, 'Instrument': df.loc[test_idx, 'Instrument'].values, 'TargetReturn_t': y_test, yhat_col_name: preds_oos}
             results_list.append(pd.DataFrame(window_predictions))
        # Store metrics for this window
        model_metrics[model_name]['oos_r2'].append(r2_oos)
        model_metrics[model_name]['optim_lambda1'].append(optim_lambda1)
        model_metrics[model_name]['optim_lr'].append(optim_lr)
        model_metrics[model_name]['is_r2_train_val'].append(r2_is)

        # Update last window info (only save valid indices and hyperparameters)
        if not train_idx.empty and not val_idx.empty and pd.notna(optim_lambda1) and pd.notna(optim_lr):
             last_train_idx = train_idx.copy(); last_val_idx = val_idx.copy()
             last_optim_lambda1 = optim_lambda1; last_optim_lr = optim_lr

        print(f"  Vindu {window_num} ({model_name}) fullført på {(datetime.datetime.now() - window_start_time).total_seconds():.1f}s. Opt L1: {optim_lambda1:.6f}, Opt LR: {optim_lr:.4f}, Win OOS R2: {r2_oos:.4f}, IS R2: {r2_is:.4f}")
        K.clear_session() # Clear TF session after each yearly window to free memory

    # --- Aggregate Results & Overall Analysis ---
    if not results_list: print(f"\nFEIL: Ingen OOS resultater for {run_label}, {model_name}."); return np.nan, None, model_metrics, (None,)*6, (None, None)
    results_df = pd.concat(results_list).reset_index(drop=True)
    print(f"\n--- Samlet Resultatanalyse ({run_label}, {model_name}, Årlig Refit) ---")
    # Calculate overall OOS R2 using Gu et al. definition
    y_true_all=results_df['TargetReturn_t']; y_pred_all=results_df[yhat_col_name]
    valid_idx_all = y_true_all.notna() & y_pred_all.notna() & np.isfinite(y_true_all) & np.isfinite(y_pred_all)
    y_t_valid_all = y_true_all[valid_idx_all]; y_p_valid_all = y_pred_all[valid_idx_all]
    R2OOS_overall = np.nan
    if len(y_t_valid_all) > 1:
        ss_res_all = np.sum((y_t_valid_all - y_p_valid_all)**2)
        ss_true_sq_all = np.sum(y_t_valid_all**2); # Denominator is sum of squared actuals
        if ss_true_sq_all > 1e-15: R2OOS_overall = 1 - (ss_res_all / ss_true_sq_all)
    # Calculate average R2 across the yearly windows
    avg_yearly_window_r2 = np.nanmean(model_metrics[model_name]['oos_r2'])
    print(f"Overall OOS R² ({run_label}, {model_name}, Gu et al. Def): {R2OOS_overall:.6f}")
    print(f"Average Yearly Window OOS R² ({run_label}, {model_name}):  {avg_yearly_window_r2:.6f}")
    model_metrics[model_name]['oos_r2_overall_gu'] = R2OOS_overall # Store overall R2

    # --- Detailed Portfolio Analysis (Using GLM+H version's function) ---
    portfolio_tables = perform_detailed_portfolio_analysis(
        results_df, df, # Pass aggregated results and the cleaned df
        benchmark_file=benchmark_file,
        ff_factor_file=ff_factor_file,
        filter_small_caps=filter_portfolio_construction,
        model_name_label=model_name # Pass specific NN name (e.g., "NN1")
    )

    # --- Analyze Prespecified Portfolios (Placeholder) ---
    prespec_r2_table, prespec_timing_table = analyze_prespecified_portfolios(
        results_df, df,
        portfolio_definitions_file=portfolio_defs_file,
        model_name_label=model_name
    )

    # --- Variable Importance (Run ONCE for 'all' subset using last window's info) ---
    vi_df = None
    if data_subset == 'all':
        print(f"\n--- Forbereder Variabel Viktighet Analyse ({model_name}) ---")
        if last_train_idx is not None and last_val_idx is not None and pd.notna(last_optim_lambda1) and pd.notna(last_optim_lr):
            # *** Run Variable Importance using last window's data and optimal hypers ***
            vi_df = calculate_variable_importance(
                df, nn_features, 'TargetReturn_t', last_train_idx, last_val_idx,
                nn_config=nn_config,             # Pass the current NN's config
                optimal_lambda1=last_optim_lambda1, # Use last optimal lambda
                optimal_lr=last_optim_lr,           # Use last optimal LR
                epochs=epochs, batch_size=batch_size, ensemble_size=ensemble_size
             )
        else: print("  Kan ikke kjøre VI: mangler siste vindus indekser eller optimale hyperparametre var NaN.")
    else: print(f"  Hopper over VI for subset '{data_subset}' (kjøres kun for 'all').")

    # --- NO Complexity Plot for NN ---

    # --- Final Summary ---
    end_time = datetime.datetime.now(); print(f"\n--- Kjøring ({run_label}, {model_name}, Årlig Refit) fullført ---"); print(f"Sluttid: {end_time.strftime('%Y-%m-%d %H:%M:%S')} (Total tid: {(end_time - start_time)})"); print(f"{'='*70}")
    # Return all relevant results
    return R2OOS_overall, vi_df, model_metrics, portfolio_tables, (prespec_r2_table, prespec_timing_table)


# --------------------------------------------------------------------------
# Main Execution Block (Orchestrates NN1-NN5 runs with Yearly Refitting)
# --------------------------------------------------------------------------
if __name__ == "__main__":
    # --- CONFIGURATION ---
    data_file = "Cleaned_OSEFX_Market_Macro_Data.csv" # <--- SET YOUR INPUT DATA FILE
    benchmark_csv_file = None # r"path/to/your/OSEBX_data.csv" # <--- UPDATE IF USING
    ff_factor_csv_file = None # r"path/to/your/F-F_Factors_plus_Mom.csv" # <--- UPDATE IF USING
    portfolio_defs_csv_file = None # r"path/to/your/portfolio_definitions_weights.csv" # <--- UPDATE IF USING

    # --- Yearly Split Parameters (Ensure they match your desired setup) ---
    INITIAL_TRAIN_YEARS = 9   # Example: 9 years (e.g., 1995-2003 if data starts 1995)
    VALIDATION_YEARS = 6    # Example: 6 years (e.g., 2004-2009)
    TEST_YEARS_PER_WINDOW = 1 # Test one year at a time (e.g., 2010, 2011, ...)

    # --- Analysis Settings ---
    filter_small_caps_portfolio = False # Set to True to filter small caps during portfolio construction
    TOP_N_FIRMS = 1000 # Definition of 'big' firms subset
    BOTTOM_N_FIRMS = 1000 # Definition of 'small' firms subset

    # --- Output Base Directory ---
    output_base_dir = "NN_Results_YearlyRefit" # Base directory for all NN results

    # --- NN Hyperparameters (Search Space from Table A.5) ---
    NN_PARAM_GRID = {
        'lambda1': [1e-5, 1e-4, 1e-3], # L1 penalties to try
        'learning_rate': [0.001, 0.01]   # Learning rates to try
    }
    # Fixed NN parameters from Table A.5
    EPOCHS = 100
    BATCH_SIZE = 10000 # Note: Very large batch size specified
    PATIENCE = 5 # For early stopping during tuning
    ENSEMBLE_SIZE = 10

    # --- Define NN Architectures (NN1 to NN5) ---
    NN_CONFIGS = {
        'NN1': {'name': 'NN1', 'hidden_units': [32]},
        'NN2': {'name': 'NN2', 'hidden_units': [64, 32]},
        'NN3': {'name': 'NN3', 'hidden_units': [96, 64, 32]},
        'NN4': {'name': 'NN4', 'hidden_units': [128, 96, 64, 32]},
        'NN5': {'name': 'NN5', 'hidden_units': [128, 96, 64, 32, 16]},
    }

    # --- Check File Existence ---
    if not os.path.exists(data_file): print(f"FEIL: Input datafil ikke funnet på '{data_file}'. Avslutter."); exit()
    if benchmark_csv_file and not os.path.exists(benchmark_csv_file): print(f"ADVARSEL: Benchmark fil ikke funnet på '{benchmark_csv_file}'. Benchmark-sammenligning deaktivert."); benchmark_csv_file = None
    if ff_factor_csv_file and not os.path.exists(ff_factor_csv_file): print(f"ADVARSEL: FF+Mom faktorfil ikke funnet på '{ff_factor_csv_file}'. Faktorregresjoner deaktivert."); ff_factor_csv_file = None
    if portfolio_defs_csv_file and not os.path.exists(portfolio_defs_csv_file): print(f"ADVARSEL: Porteføljedefinisjonsfil ikke funnet på '{portfolio_defs_csv_file}'. Analyse av prespesifiserte porteføljer deaktivert."); portfolio_defs_csv_file = None
    if not os.path.exists(output_base_dir): os.makedirs(output_base_dir); print(f"Opprettet base mappe: {output_base_dir}")

    # --- RUN ANALYSIS FOR EACH NN and SUBSET ---
    all_results_r2_summary = defaultdict(dict) # Store R2[subset][nn_name]
    all_variable_importance = {} # Store VI[nn_name] for 'all' subset
    all_model_metrics = defaultdict(dict) # Store detailed metrics[subset][nn_name]
    all_portfolio_tables = defaultdict(dict) # Store portfolio tables[subset][nn_name] -> (decile_ew, decile_vw, hl_ew, hl_vw, long_ew, long_vw)
    all_prespecified_results = defaultdict(dict) # Store prespecified tables[subset][nn_name] -> (prespec_r2, prespec_timing)

    # --- Loop through each NN configuration ---
    for nn_name, nn_conf in NN_CONFIGS.items():
        print(f"\n{'#'*30} Starting Analysis for {nn_name} (Yearly Refit) {'#'*30}")
        # Create specific output directory for this NN model
        nn_output_dir = os.path.join(output_base_dir, nn_name)
        if not os.path.exists(nn_output_dir): os.makedirs(nn_output_dir); print(f"Opprettet mappe: {nn_output_dir}")

        # --- Loop through each data subset ---
        for subset in ['all', 'big', 'small']:
            # Run the full analysis pipeline for this NN and subset
            r2, vi_df_run, metrics_run, port_tables_run, prespec_tables_run = run_analysis_for_subset(
                file_path=data_file,
                data_subset=subset,
                benchmark_file=benchmark_csv_file,
                ff_factor_file=ff_factor_csv_file,
                portfolio_defs_file=portfolio_defs_csv_file,
                filter_portfolio_construction=filter_small_caps_portfolio,
                top_n=TOP_N_FIRMS, bottom_n=BOTTOM_N_FIRMS,
                nn_config=nn_conf, # Pass the specific NN config for this iteration
                param_grid_nn=NN_PARAM_GRID, # Pass the hyperparameter grid to search
                epochs=EPOCHS, batch_size=BATCH_SIZE, patience=PATIENCE, ensemble_size=ENSEMBLE_SIZE,
                initial_train_years=INITIAL_TRAIN_YEARS, # Use yearly params
                val_years=VALIDATION_YEARS,
                test_years=TEST_YEARS_PER_WINDOW
            )

            # Store results keyed by subset and NN name
            all_results_r2_summary[subset][nn_name] = r2
            all_model_metrics[subset][nn_name] = metrics_run.get(nn_name, {}) # Get metrics specific to this NN run
            all_portfolio_tables[subset][nn_name] = port_tables_run # Store tuple of 6 portfolio tables
            all_prespecified_results[subset][nn_name] = prespec_tables_run # Store tuple of 2 prespecified tables
            # Store Variable Importance results only if calculated (subset='all')
            if subset == 'all' and vi_df_run is not None:
                 all_variable_importance[nn_name] = vi_df_run

            # --- Save portfolio tables immediately for this subset/NN run ---
            if port_tables_run and len(port_tables_run) == 6:
                # Names corresponding to the tables returned by perform_detailed_portfolio_analysis
                table_names = ['decile_ew', 'decile_vw', 'hl_risk_ew', 'hl_risk_vw', 'long_risk_ew', 'long_risk_vw']
                for i, table_df in enumerate(port_tables_run):
                    if table_df is not None and not table_df.empty:
                        filename = os.path.join(nn_output_dir, f"portfolio_{subset}_{table_names[i]}_yearly.csv")
                        try: table_df.to_csv(filename); print(f" -> Porteføljetabell lagret: {filename}")
                        except Exception as e: print(f"  FEIL ved lagring av porteføljetabell {filename}: {e}")

            # --- Save prespecified portfolio tables (if generated and not empty) ---
            if prespec_tables_run and len(prespec_tables_run) == 2:
                 names_prespec = ['R2oos_prespecified', 'TimingGains_prespecified']
                 for i, table_df in enumerate(prespec_tables_run):
                      if table_df is not None and not table_df.empty:
                           filename = os.path.join(nn_output_dir, f"prespecified_{subset}_{names_prespec[i]}_yearly.csv")
                           try: table_df.to_csv(filename); print(f" -> Prespesifisert tabell lagret: {filename}")
                           except Exception as e: print(f"  FEIL ved lagring av prespesifisert tabell {filename}: {e}")


        # --- Save VI results for this NN (if calculated from 'all' subset) ---
        if nn_name in all_variable_importance:
             print(f"\nLagrer Variabel Viktighet resultater for {nn_name} (fra siste vindu)...")
             try:
                  # Save the VI dataframe calculated after the loop for the 'all' subset
                  vi_filename = os.path.join(nn_output_dir, "variable_importance_last_window_yearly.csv")
                  all_variable_importance[nn_name].to_csv(vi_filename, index=False)
                  print(f" -> Variabel Viktighet ({nn_name}) lagret til {vi_filename}")
             except Exception as e: print(f"  FEIL ved lagring av VI for {nn_name}: {e}")

    # --- Print Final Consolidated R2 Summary Table ---
    print("\n\n" + "="*30 + " Final Consolidated OOS R2 Summary (NNs - Yearly Refit) " + "="*30)
    # Reorganize data for DataFrame: Index=Subset, Columns=NN1, NN2,...
    summary_r2_for_df = defaultdict(dict)
    for subset, nn_results in all_results_r2_summary.items():
        for nn_name_key, r2_val in nn_results.items():
            summary_r2_for_df[subset][nn_name_key] = r2_val

    r2_summary_df = pd.DataFrame.from_dict(summary_r2_for_df, orient='index')
    r2_summary_df.index.name = "Subset"
    # Ensure columns are ordered NN1, NN2, ...
    nn_cols_order = [f'NN{i}' for i in range(1, 6) if f'NN{i}' in r2_summary_df.columns]
    r2_summary_df = r2_summary_df[nn_cols_order]
    # Format as percentage points for printing
    r2_summary_df_pct = r2_summary_df * 100
    print(r2_summary_df_pct.round(4)) # Display R2 * 100
    try: # Save consolidated summary table (as original R2 values, not percentages)
        r2_summary_filename = os.path.join(output_base_dir, "nn_consolidated_r2_summary_yearly.csv")
        r2_summary_df.to_csv(r2_summary_filename)
        print(f"\n -> Konsolidert R2 Sammendrag lagret til {r2_summary_filename}")
    except Exception as e: print(f"  FEIL ved lagring av Konsolidert R2 Sammendrag: {e}")
    print("="*78)


    print("\nFull NN analyse (NN1-NN5 - Årlig Refitting) fullført.")